In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import h5py
import numpy as np
from tqdm import tqdm
import pandas as pd
import os

In [62]:
# ----------------------------
# Dataset定義（PCam用）
# ----------------------------
class PCamDataset(Dataset):
    def __init__(self, h5_x_path, h5_y_path=None, transform=None):
        self.x_path = h5_x_path
        self.y_path = h5_y_path
        self.transform = transform
        self.has_labels = h5_y_path is not None  # ← 修正

        with h5py.File(h5_x_path, 'r') as x_file:
            self.length = len(x_file['x'])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        with h5py.File(self.x_path, 'r') as x_file:
            image = x_file['x'][idx]

        image = transforms.ToPILImage()(image.astype(np.uint8))

        if self.transform:
            image = self.transform(image)

        if self.has_labels:
            with h5py.File(self.y_path, 'r') as y_file:
                label = y_file['y'][idx].astype(np.float32)
            return image, label
        else:
            return image


In [63]:
# ----------------------------
# データ変換
# ----------------------------
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(96, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [64]:
# ----------------------------
# データローダ-
# ----------------------------
train_dataset = PCamDataset('camelyonpatch_level_2_split_train_x.h5',
                          'camelyonpatch_level_2_split_train_y.h5',
                          transform=transform_train)

val_dataset = PCamDataset('valid_x_uncompressed.h5',
                         'valid_y_uncompressed.h5',
                         transform=transform_val_test)

test_dataset = PCamDataset('camelyonpatch_level_2_split_test_x.h5',
                         'camelyonpatch_level_2_split_test_y.h5',  # テストラベルを追加
                         transform=transform_val_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [65]:
import h5py
import numpy as np

with h5py.File('camelyonpatch_level_2_split_train_y.h5', 'r') as f:
    labels = f['y'][:]
    unique, counts = np.unique(labels, return_counts=True)
    print(dict(zip(unique, counts)))

with h5py.File('valid_y_uncompressed.h5', 'r') as f:
    labels = f['y'][:]
    unique, counts = np.unique(labels, return_counts=True)
    print(dict(zip(unique, counts)))

with h5py.File('camelyonpatch_level_2_split_test_y.h5', 'r') as f:
    labels = f['y'][:]
    unique, counts = np.unique(labels, return_counts=True)
    print(dict(zip(unique, counts)))


{np.uint8(0): np.int64(131072), np.uint8(1): np.int64(131072)}
{np.uint8(0): np.int64(16399), np.uint8(1): np.int64(16369)}
{np.uint8(0): np.int64(16391), np.uint8(1): np.int64(16377)}


In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [67]:
# ----------------------------
# Focal Loss 定義
# ----------------------------
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, input, target):
        bce_loss = self.bce(input.view(-1), target.view(-1))
        pt = torch.exp(-bce_loss)
        loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return loss.mean() if self.reduction == 'mean' else loss.sum()

In [68]:

# ----------------------------
# ResNet50 モデル構築
# ----------------------------
def get_model():
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
    return model.to(device)

In [69]:
# ----------------------------
# モデルの評価関数
# ----------------------------
def evaluate(model, loader):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            preds.extend(torch.sigmoid(outputs).cpu().numpy() > 0.5)
            targets.extend(labels.cpu().numpy())

    return accuracy_score(targets, preds)

In [70]:
def train_model(model, train_loader, val_loader, num_epochs=30, lr=1e-4, save_path='best_model.pth'):
    criterion = FocalLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        preds, targets = [], []

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            # 型変換を明示的に行う
            probas = torch.sigmoid(outputs).detach().cpu().numpy()
            predictions = (probas > 0.5).astype(np.int64)  # int64に統一
            labels_np = labels.cpu().numpy().astype(np.int64)  # int64に統一

            preds.extend(predictions.tolist())
            targets.extend(labels_np.tolist())

        scheduler.step()
        train_acc = accuracy_score(targets, preds)
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader.dataset):.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(" Best model saved!")

In [71]:
def test(model, loader):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            # Ensure proper squeezing (handles batch size = 1 case)
            if outputs.dim() > 1:
                outputs = outputs.squeeze()
            
            # Convert to numpy with explicit type
            probas = torch.sigmoid(outputs).cpu().numpy()
            predictions = (probas > 0.5).astype(np.int64)  # Changed to int64
            labels_np = labels.cpu().numpy().astype(np.int64)  # Changed to int64

            # Verify shapes match
            assert predictions.shape == labels_np.shape, \
                   f"Shape mismatch: preds {predictions.shape}, targets {labels_np.shape}"
            
            preds.extend(predictions.tolist())
            targets.extend(labels_np.tolist())

    # Additional validation
    print(f"Sample predictions: {preds[:10]}")
    print(f"Sample targets: {targets[:10]}")
    print(f"Unique predictions: {np.unique(preds)}")
    print(f"Unique targets: {np.unique(targets)}")

    # Calculate metrics
    acc = accuracy_score(targets, preds)
    cm = confusion_matrix(targets, preds)
    
    print(f"\nTest Accuracy: {acc:.4f}")
    print("Confusion Matrix:\n", cm)
    
    # Additional metrics
    if len(np.unique(targets)) == 2:  # Binary classification
        print("\nClassification Report:")
        print(classification_report(targets, preds, target_names=['Class 0', 'Class 1']))
    
    return acc, cm

In [72]:
# ----------------------------
# 実行
# ----------------------------
if __name__ == '__main__':
    model = get_model()
    train_model(model, train_loader, val_loader, num_epochs=30, lr=1e-4)

    model.load_state_dict(torch.load('best_model.pth'))
    test(model, test_loader)


Epoch 1/30:  17%|█▋        | 695/4096 [00:43<03:30, 16.14it/s]


KeyboardInterrupt: 